# Welcome to plotAR

In [ ]:
import plotar
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import requests
import json

## Iris

First get the Iris data from scikit-learn

In [ ]:
from sklearn import datasets
iris = datasets.load_iris()

_IRIS_ - the hello-world of statistics - consists of 150 samples in 3 species (in `iris.target`) and the features are dimensions of the indivicual blooms (`sepal/petal` `length/width`).

We plot the first three features and give the species as colors (and give the Plot a name)

In [ ]:
plot = plotar.plotar(iris.data, iris.target, name='iris')

This wrote the plot to the server - how does it look like?

In [ ]:
plot

Just scan the QR-Code with your mobile device and open the URL - this will open this in your browser. Tap on the the **AR-Icon** in the Modelview to step in the Augmented Reality or take the link **Open in VR** to your VR googles.

Alternatively you can save the plot in it's JSON-format or in a rendered 3D format (glTF for Android, USDZ for iOS/macOS, any of those for Blender):

In [ ]:
plot.write("examples/iris.json", format="json")
plot.write("examples/iris.gltf", format="gltf")
plot.write("examples/iris.usdz", format="usdz")

## Architecture

What should happen here is, that your mobile device connects to this very Jupyter server! For that to work plotar tries to guess a URL that also works from you mobile: ![Architecture](images/architecture.png)

**Troubleshooting**: If you cannot connect to this server consider the following steps:
* Enable connections from outside by (re-)starting Jupyter: `jupyter lab --ip="*"`
* Try to have your mobile device to be on the same network as your desktop.

**WARNING** Jupyter is secured by default to have a non-guessable token to get some level of security, but still you probably do not use HTTPS, so anybody intercepting the traffic between you mobile device and your desktop can see all your data! This might be ok in your home network or in a company enterprise - be cautious!

Traffic with mybinder.org actually is secured by HTTPS and the token.

## GAPminder

Now take the GAPminder data: development indicators for all countries in the world.

First the Plotly-Version that has only data for years 1952 to 2007:

In [ ]:
url = 'https://raw.githubusercontent.com/plotly/datasets/master/gapminderDataFiveYear.csv'
gap = pd.read_csv(url)

In [ ]:
gap

To keep it visible let's take only the European countries:

In [ ]:
plotar.linear(gap.query("continent=='Europe'"), xyz=['gdpPercap','year','lifeExp'],
              col='country', size='pop')

In [ ]:
plot.write("examples/gapminder.json", format="json gltf glb usda usdz".split())

## GAPminder animated

Now let's animate it like in Hans Rosling's famours [Lecture](https://www.youtube.com/watch?v=jbkSRLYSojo) showing how world wars etc. and general development shaped countries by looking at ther income per capita, life expectancy and population as size  - however we can use one more dimension! children_per_woman are available for the whole time period.

For this we actually want to have more historic data - thanks to University of Toronto:

In [ ]:
url = 'https://github.com/UofTCoders/2018-09-10-utoronto/raw/gh-pages/data/world-data-gapminder.csv'
gap = pd.read_csv(url)
gap

In [ ]:
plot = plotar.animate(gap.query("region=='Europe'"), xyz=['income','children_per_woman','life_expectancy'],
    group='country', col='sub_region', size='population', animation_frame='year',
    name="gapminder-animated")
plot

The spheres are nice, however we don't know which dot is which country - so take the country name directly to the plot:

In [ ]:
plot.write("examples/gapminder-animated.json", format="json gltf glb usda usdz".split())

In [ ]:
plot = plotar.animate(gap.query("region=='Europe'"), xyz=['income','children_per_woman','life_expectancy'],
    group='country', col='sub_region', size='population', animation_frame='year',
    label = 'country', name="gapminder-animated-label")
plot

In [ ]:
plot.write("examples/gapminder-animated-label.json", format="json gltf glb usda usdz".split())

## D ONE Team

We scrape the http://d-one.ai/team webpage and extract some features on the team member's description

In [ ]:
from bs4 import BeautifulSoup

In [ ]:
url = 'https://d-one.ai/team'
res = requests.get(url)

In [ ]:
pd.set_option("max_colwidth", 400)

In [ ]:
soup = BeautifulSoup(res.text, 'html.parser')
x = soup.find_all("div", class_="details")
team = pd.DataFrame( dict(name=_.find_all('h3')[0].text, text=_.find_all('p')[0].text) for _ in x )
team = team.drop_duplicates('name')
team

Extract two approximate counts: number of words and number of sentence (the latter actuall fails e.g. if many Abbreviations are used :-| )

In [ ]:
team['n_sent'] = team.text.str.replace(r'[^.]','', regex=True).str.len()
team['n_word'] = team.text.str.replace(r'[^ ]','', regex=True).str.len()+1

Extract the last mentioned year - usually that is, when people started. If we do not find one, take 2000 as a default value - that is before the company was founded!

In [ ]:
years = team.text.apply(lambda x: ([2000] + [_ for _ in x.split() if _.startswith("20")])[-1])
team['year_start'] = years.astype(str).str.rstrip('.').astype(int)
team['year_start'].value_counts(dropna=False)

In [ ]:
team['dr'] = team.name.str.startswith("Dr.")
team['dr'].value_counts(dropna=False)

In [ ]:
team

In [ ]:
team.columns

In [ ]:
plot = plotar.plotar(team, xyz=['n_word', 'n_sent', 'year_start'], col='dr', label='name', size=0.2)
plot

## CH - surface of Switzerland

We use the [Swisstopo Digital Height Model](https://www.swisstopo.admin.ch/de/geodata/height/dhm25200.html) 200m grid to draw a surface of Swizterland.

In [ ]:
url = 'https://data.geo.admin.ch/ch.swisstopo.digitales-hoehenmodell_25/data.zip'
file_name = 'DHM200.asc'

Download Zip file, unzip the part we need to file:

In [ ]:
def get_or_download(url, file_name, cache="tmp"):
    file = Path(cache) / file_name
    if not file.exists():
        from io import BytesIO
        from zipfile import ZipFile
        import shutil
        print(f"Downloading {url} to {file} ...")
        zipfile = ZipFile(BytesIO(requests.get(url).content))
        with open(file, 'wb') as f:
            shutil.copyfileobj(zipfile.open(file_name), f)
        print(f"Downloaded {file} from {url}")
    else:
        print(f"getting {file} from cache")
    return file

In [ ]:
file = get_or_download(url, file_name)

The GeoSpatial Information is in the first 6 rows of the file:

In [ ]:
%%time
y_head = {k: float(v) for k,v in np.genfromtxt(file, dtype=str, max_rows=6)}
print(y_head)
y = np.genfromtxt(file, skip_header=6, skip_footer=1)
y.shape

In [ ]:
n,m = [int(y_head[_]) for _ in ['NCOLS','NROWS']]
n,m

In [ ]:
img = y.flatten()[:n*(m-1)].reshape((m-1,n))

This file is actually rather big - if you want you can make it smaller by setting factor to e.g. 5, 10, 20. We set it to 2 for mybinder.org

In [ ]:
factor = 2

In [ ]:
img = img[::factor,::factor]

In [ ]:
xvec = np.arange(img.shape[1]) * y_head['CELLSIZE'] * factor
yvec = np.arange(img.shape[0]) * y_head['CELLSIZE'] * factor

Impute negative, i.e. NA values to some level below switerlands elevation

In [ ]:
img[img>0].min()

In [ ]:
img[img<0] = 150

Quickly draw it here so we understand whats happening:

In [ ]:
plt.imshow(img, interpolation='none');

Actually - since the Swiss geographical coordinate system (LV95) is in meters (our xvec and yvec), and the height is as well - this is in all our export formats a correct scale representation of the surface of switerzland!

In the plots obviously it will be shown on a much smaller scale.

In [ ]:
plot = plotar.surfacevr(img, x=xvec, y=yvec, auto_scale=False, name="CH")
plot

## CH-color - surface of Switzerland with color

Now add the official satellite image on top of that surface.

In [ ]:
landsat_file = get_or_download("https://data.geo.admin.ch/ch.swisstopo.images-landsat25/data.zip", "LandsatMos25.tif")
landsat_metadata_file = get_or_download("https://data.geo.admin.ch/ch.swisstopo.images-landsat25/data.zip", "Landsatmos25.TFW")

In [ ]:
y_head

In [ ]:
sat_head = np.genfromtxt(landsat_metadata_file).astype(np.int64)
sat_head.T

Description [[Source](http://www.omg.unb.ca/~jonnyb/processing/geotiff_tifw_format.html)]:
* First row is x-pixel resolution
* Second and third rows are so-called "rotational components" but are set to zero in the case of an unrotated mapsheet.
* The fourth row is the y-pixel resolution. The negative sign indicates that the image y-axis is positive down which is the opposite from real world coordinates.
* The 5th and 6th rows are the Easting and Northing of the upper left pixel (0,0 in image coordinates). 

If you compare `y_head` and `sat_head` you see that unfortunately we need to crop the satellite to match the frame of the surface data:

In [ ]:
crop = (
    -sat_head[4] + (y_head['XLLCORNER']),
    sat_head[5] - (y_head['YLLCORNER'] + y_head['CELLSIZE'] * y_head['NROWS']),
)
crop = crop + (
    crop[0] + y_head['CELLSIZE'] * y_head['NCOLS'],
    crop[1] + y_head['CELLSIZE'] * y_head['NROWS'],
)
np.array(crop)/25.0

In [ ]:
from PIL import Image

In [ ]:
landsat = Image.open(landsat_file)

No crop it and rescale it to the size of the surface

In [ ]:
landsat_small = landsat.crop(np.array(crop)/25.0).resize(reversed(img.shape))

In [ ]:
landsat_small.size, np.array(landsat_small).shape, img.shape

In [ ]:
landsat_small

No plot it and resize it - also exaggerate the height by a factor ~3

In [ ]:
plot = plotar.surfacevr(img/100000, x=xvec/300000, y=yvec/300000, surfacecolor=np.array(landsat_small).astype(int).tolist(),
                             auto_scale=False, name="CH-color",)

In [ ]:
plot.write("examples/CH-color.json", format="json gltf glb usda usdz".split())

## Planets

We visualize the position of the Planets in the solar system at some time using the skyfield package.

In [ ]:
from skyfield.api import Loader
import io

In [ ]:
load = Loader("./tmp/")

In [ ]:
ts = load.timescale()
t = ts.utc(2022, 5, 22, 15, 19)
tarr = ts.utc(2022, 5, range(-365,365, 14))

**Note:** on mybinder.org unfortunately ftp-downloads are blocked so this will run into a timeout. We are preparing a workaround.

In [ ]:
planets = load('de421.bsp')  # ephemeris DE421

In [ ]:
planet_names = [ _[-1] for i,_ in planets.names().items() if 0 < i < 100 ]
print(len(planet_names))
planet_names

https://en.wikipedia.org/wiki/Planet

In [ ]:
_ = """i	Name	Equatorial diameter [i]	Mass [i]	Semi-major axis (AU)	Orbital period (years)	Inclination to Sun's equator (°)	Orbital eccentricity	Rotation period (days)	Confirmed moons	Axial tilt (°)	Rings	Atmosphere
1.	Mercury	0.383	0.06	0.39	0.24	3.38	0.206	58.65	0	0.10	no	minimal
2.	Venus	0.949	0.81	0.72	0.62	3.86	0.007	−243.02	0	177.30	no	CO2, N2
3.	Earth	1.000	1.00	1.00	1.00	7.25	0.017	1.00	1	23.44	no	N2, O2, Ar
4.	Mars	0.532	0.11	1.52	1.88	5.65	0.093	1.03	2	25.19	no	CO2, N2, Ar
5.	Jupiter	11.209	317.83	5.20	11.86	6.09	0.048	0.41	79	3.12	yes	H2, He
6.	Saturn	9.449	95.16	9.54	29.45	5.51	0.054	0.44	82	26.73	yes	H2, He
7.	Uranus	4.007	14.54	19.19	84.02	6.48	0.047	−0.72	27	97.86	yes	H2, He, CH4
8.	Neptune	3.883	17.15	30.07	164.79	6.43	0.009	0.67	14	29.60	yes	H2, He, CH4j"""
planet_info = pd.read_csv(io.StringIO(_), delimiter='\t').drop(columns=['i']).set_index("Name")
planet_info

In [ ]:
planet_info['Equatorial diameter [i]']

In [ ]:
planets_traj_xyz = pd.concat([
    pd.DataFrame(planets[_].at(tarr).ecliptic_xyz().au.T, columns=list('xyz'))
    .assign(planet=_).assign(t=tarr.tt.astype(int))
    for _ in planet_names
])
planets_traj_xyz

In [ ]:
#%debug
plot = plotar.animate(planets_traj_xyz, xyz=['x','y','z'],
              group='planet', col='planet', size=planet_info['Equatorial diameter [i]'].to_list()+[1,1],
              animation_frame='t', name='planets')
plot

In [ ]:
plot.write("examples/planets.json", format="json gltf glb usda usdz".split())

## Flights

In [ ]:
from skyfield.api import N, W, wgs84, load
from skyfield.functions import length_of

In [ ]:
url = 'https://www.flightradar24.com/flights/most-tracked'
# flightradar24 refuses 'User-Agent': 'python-requests/2.25.1' with error 451 Unavailable For Legal Reasons
res = requests.get(url, headers={'User-Agent': ''})

In [ ]:
most_tracked = pd.DataFrame(res.json()['data'])
most_tracked['name'] = most_tracked.fillna('_').apply(
    lambda _: f"{_.callsign} {_.from_city}->{_.to_city}", axis=1)
most_tracked

In [ ]:
def get_flight(flight_id, name):
    url = f'https://data-live.flightradar24.com/clickhandler/?version=1.5&flight={flight_id}'
    res = requests.get(url)
    trail = pd.DataFrame(res.json()['trail'])
    trail['flight_id'] = flight_id
#     trail['name'] = name
    return trail

In [ ]:
flights = pd.concat(( get_flight(_.flight_id, _.name) for _ in most_tracked.itertuples()), )

In [ ]:
trail = most_tracked.merge(flights, on="flight_id")
trail

In [ ]:
_ = trail.apply(lambda _: wgs84.latlon(_.lat, _.lng, _.alt*10).at(t).position.m, axis=1)
trail[['x','y','z']] = np.stack(_) / 1000
trail

In [ ]:
plot = plotar.linear(trail, xyz=['x','y','z'], col='name', size=trail.spd/10, auto_scale=True, type='l', name='flights')
plot

## Lorenz Attractor

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
rho = 28.0
sigma = 10.0
beta = 8.0 / 3.0

def f(state, t):
    x, y, z = state  # Unpack the state vector
    return sigma * (y - x), x * (rho - z) - y, x * y - beta * z  # Derivatives

state0 = np.array([1.0, 1.0, 1.0])
t = np.arange(0.0, 40.0, 0.01)

states = odeint(f, state0, t)

In [ ]:
states.shape

In [ ]:
plot = plotar.linear(states, auto_scale=True, type='l', name="lorenz")
plot

In [ ]:
plot.write("examples/lorenz.json", format="json gltf glb usda usdz".split())

## SOLA 2022

The [Sola 2022](https://trackmaxx.ch/maps/?m=ec368d93-aff4-4a7e-b0a5-24b7f9683a32&style=swisstopo&legend=full&tracks=1,2,3,4,5,6,8,7,9,10,11,12,13,14,20&labels=iconsubergabebuchlern,iconsuebergaben,icstrecke10,icstrecke11,icstrecke14,icstrecke2,icstrecke3,icstrecke4,icstrecke5,icstrecke6,icstrecke7,icstrecke8,icugbucheggplatz,icstrecke9,icugegg,icugfelsenegg,icugfluntern,icugforch,icughoenggerberg,icugirchel,icuguetliberg,icugwitikon,icugzumikon&h=8d68)

In [ ]:
from bs4 import BeautifulSoup

In [ ]:
url = "https://tmxx-static.s3.amazonaws.com/ous/asvzsolazh/mapstudio/gpx/strecke01.gpx"
res = requests.get(url)

In [ ]:
pd.set_option("max_colwidth", 400)

In [ ]:
soup = BeautifulSoup(res.text, 'html.parser')

In [ ]:
def sola_track(strecke):
    url = f"https://tmxx-static.s3.amazonaws.com/ous/asvzsolazh/mapstudio/gpx/strecke{strecke:02}.gpx"
    res = requests.get(url)
    soup = BeautifulSoup(res.text, 'html.parser')
    x = soup.find_all("trkpt")
    track = pd.DataFrame( _.attrs for _ in x )
    track['ele'] = pd.Series( _.ele.text for _ in x)
    track = track.astype(float)
    track['strecke'] = strecke
    return track
sola_track(1)

In [ ]:
sola = pd.concat( (sola_track(_) for _ in range(1,15)), axis=0)
sola

In [ ]:
sola.plot.line('lon','lat',);

In [ ]:
plot = plotar.linear(sola, xyz=['lon','lat','ele'], col=sola.strecke.astype(str), auto_scale=True, type='l', name="sola")
plot